## part1

In [1]:
## 关于题目，有两个例子不是很好理解
# == Pair 7 ==
# - Compare [[[]]] vs [[]]
#   - Compare [[]] vs []
#     - Right side ran out of items, so inputs are not in the right order
    
# 逐层解析后，左侧len([[]])为1， 右侧len([])为0，题目说右侧run out of items

# == Pair 2 ==
# - Compare [[1],[2,3,4]] vs [[1],4]
#   - Compare [1] vs [1]
#     - Compare 1 vs 1
#   - Compare [2,3,4] vs 4
#     - Mixed types; convert right to [4] and retry comparison
#     - Compare [2,3,4] vs [4]
#       - Compare 2 vs 4
#         - Left side is smaller, so inputs are in the right order 

# 最后一步，Compare [2,3,4] vs [4]难道不是右侧更先run out of吗？


## 为了能继续做下去我暂时忽略了这里的问题，约定为:
# 1. 两个list比较，以len较小者的len为主，比较0:len的元素，只要前len个元素符合左小右大的条件就行；
#     如果完全一致，再根据len判断，要求len（left）< len(right);
# 2. 至于pair2的例子，[]里没有任何元素可以和[[]]里的[]相比，视为得不出结论; 最终对比len。 
# 综上，len的对比视为兜底解决方案


# 以及 题目似乎遗漏了：从左往右比较，类似数字的进制，只要left在更高的进位上比right小，就不用继续比较了。不要求各个位置上的元素都满足。

In [2]:
f = open("input.txt", 'r')
lines = f.readlines() 
lines.append(" ") ## 防止最后一对数据被遗漏
f.close()

texts = [] 

i = 1 
tmp = [] 
for line in lines:
    if line.strip() == '':
        assert len(tmp) == 2 
        texts.append((i, tmp[0], tmp[1]))

        tmp = [] 
        i += 1 
    else:
        l1 = eval(line.strip())
        tmp.append(l1)

def make_compare(left, right):
    types = (type(left), type(right))
    
    if types == (int, int):
        return (None if left == right else left < right) 
    elif types == (list, int):
        return make_compare(left, [right])
    elif types == (int, list):
        return make_compare([left], right)
    
    else:
        for x, y in zip(left, right): 
            result = make_compare(x, y) 
            if result is not None: ## 从左往右逐个元素比较，任意一步有结果就可以返回，否则继续比较；相等不是结果
                return result 
        return (None if len(left) == len(right) else len(left) < len(right))
    
ans = [] 
    
for i, left, right in texts:
    if make_compare(left, right):
        ans.append(i)
print(sum(ans))

5580


In [3]:
# 一些例子
make_compare([1, 2], [2, 0])

True

In [4]:
make_compare([1, 2], [1, 0])

False

In [5]:
## 似乎可以理解为12.3 和 12.04对比
make_compare([1, 2, [3]], [1, 2, [0, 4]])

False

## part2

In [6]:
from functools import cmp_to_key

divider1 = [[2]]
divider2 = [[6]]

## 先清洗下lines 
clean_data = []
for line in lines:
    if line.strip() != "":
        clean_data.append(eval(line.strip()))
        
clean_data.append(divider1)
clean_data.append(divider2)
   
## 对clean_data进行排序需要自定义一个key    
def compare_key(left, right):
    if make_compare(left, right) == True:
        return -1 
    elif make_compare(left, right) == False:
        return 1
    else:
        return 0
    
sorted_clean_data = sorted(clean_data, key=cmp_to_key(compare_key))

In [7]:
ans = []

for i, data in enumerate(sorted_clean_data, 1):
    if data == divider1 or data == divider2:
        ans.append(i)

assert len(ans) == 2 

print(ans[0] * ans[1])

26200
